In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import cohen_kappa_score as kappa_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.preprocessing import StandardScaler
kappa_scorer = make_scorer(kappa_score)
from keras.constraints import maxnorm
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

import glob
import cv2
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
import pandas as pd
import tensorflow as tf
from keras import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Dropout, Dense, np
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [169]:
train_df = pd.read_csv("../input/all/train.csv")
test_df = pd.read_csv("../input/all/test/test.csv")

In [170]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [171]:
cat_cols = ['Type','Age','Breed1','Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 
          'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized','Health', 'Quantity','State','PhotoAmt']

In [172]:
num_cols = ['Fee']

In [173]:
text_cols = ['Description']

## Handling categorical columns

In [174]:
embed_sizes = [len(train_df[col].unique()) + 1 for col in cat_cols]

In [175]:
print(embed_sizes)

[3, 107, 177, 136, 4, 8, 8, 7, 5, 4, 4, 4, 4, 4, 20, 15, 32]


## Handling numerical columns

In [176]:
print('scaling num_cols')
for col in num_cols:
    print('scaling {}'.format(col))
    col_mean = train_df[col].mean()
    train_df[col].fillna(col_mean, inplace=True)
    test_df[col].fillna(col_mean, inplace=True)
    scaler = StandardScaler()
    train_df[col] = scaler.fit_transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = scaler.transform(test_df[col].values.reshape(-1, 1))

scaling num_cols
scaling Fee


/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Handling text columns

In [101]:
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [41]:
print('getting embeddings')
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in tqdm(open('//Users/zekunzhao/Downloads/wiki-news-300d-1M-subword.vec')))


1106it [00:00, 11053.92it/s]

getting embeddings


1000000it [01:22, 12085.05it/s]


In [127]:
num_words = 20000
maxlen = 80
embed_size = 300

In [128]:
train_df['Description'] = train_df['Description'].astype(str).fillna('no text')
test_df['Description'] = test_df['Description'].astype(str).fillna('no text')

In [129]:
print("   Fitting tokenizer...")
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_df['Description'].values.tolist())

   Fitting tokenizer...


In [130]:
train_df['Description'] = tokenizer.texts_to_sequences(train_df['Description'])
test_df['Description'] = tokenizer.texts_to_sequences(test_df['Description'])

In [131]:
word_index = tokenizer.word_index
nb_words = min(num_words, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= num_words: continue
    try:
        embedding_vector = embeddings_index[word]
    except KeyError:
        embedding_vector = None
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [143]:
def get_input_features(df):
    X = {'description':pad_sequences(df['Description'], maxlen=maxlen)}
    X['numerical'] = np.array(df[num_cols])
    for cat in cat_cols:
        X[cat] = np.array(df[cat])
    return X

## Define NN Model

In [177]:
from keras.layers import Input, Embedding, Concatenate, Flatten, Dense, Dropout, BatchNormalization,LSTM, CuDNNLSTM, SpatialDropout1D
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPool1D
from keras.models import Model
from keras.optimizers import  Adam

def prepare_data(data):
    pet_id = data.PetID

    # Remove unused features
    data.drop(['RescuerID', 'Description', 'PetID', 'State'], axis=1, inplace=True)

    # Apply binning to ages
    data['Age'] = pd.cut(data['Age'], [-1, 2, 3, 6, 255], labels=[0, 1, 2, 3])

    # Apply binning to fee
    data['Fee'] = pd.cut(data['Fee'], [-1, 50, 100, 200, 3000], labels=[0, 1, 2, 3])

    # Apply binning to photo amount
    data['PhotoAmt'] = pd.cut(data['PhotoAmt'], [-1, 1, 5, 10, 100], labels=[0, 1, 2, 3])

    # Apply binning to video amount
    data['VideoAmt'] = pd.cut(data['VideoAmt'], [-1, 1, 100], labels=[0, 1])

    # Replace names with 1 is present, 0 if not present
    data.loc[data['Name'].notnull(), 'Name'] = 1
    data.loc[data['Name'].isnull(), 'Name'] = 0

    # Fill missing continuous data
    data_continuous = data.select_dtypes(exclude=['object'])
    data_continuous.fillna(0, inplace=True)

    # Fill missing string data
    data_categorical = data.select_dtypes(include=['object'])
    data_categorical.fillna('NONE', inplace=True)

    final_data = data_continuous.merge(data_categorical, left_index=True, right_index=True)

    return final_data, data_categorical, data_continuous, pet_id, data.shape[1]


categorical_inputs = []
for cat in cat_cols:
    categorical_inputs.append(Input(shape=[1], name=cat))

categorical_embeddings = []
for i, cat in enumerate(cat_cols):
    categorical_embeddings.append(
        Embedding(embed_sizes[i], 10)(categorical_inputs[i]))

categorical_logits = Concatenate()([Flatten()(cat_emb) for cat_emb in categorical_embeddings])
categorical_logits = Dense(256, activation = 'relu')(categorical_logits)


numerical_inputs = Input(shape=[len(num_cols)], name='numerical')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)
numerical_logits = Dense(128, activation = 'relu')(numerical_logits)

text_inp = Input(shape=[maxlen], name='description')
text_embed = Embedding(nb_words, embed_size, weights=[embedding_matrix], trainable=False)(text_inp)
text_logits = SpatialDropout1D(0.2)(text_embed)
text_logits = Bidirectional(LSTM(64, return_sequences=True))(text_logits)
avg_pool = GlobalAveragePooling1D()(text_logits)
max_pool = GlobalMaxPool1D()(text_logits)
text_logits = Concatenate()([avg_pool, max_pool])

x = Concatenate()([categorical_logits, text_logits, numerical_logits])
x = BatchNormalization()(x)
x = Dense(128, activation = 'relu')(x)
x = Dropout(0.3)(x)
out = Dense(5, activation = 'sigmoid')(x)

model = Model(inputs=[text_inp] + categorical_inputs + [numerical_inputs],outputs=out)
model.compile(optimizer=Adam(lr = 0.0001), loss = 'mse')


In [178]:
from sklearn.model_selection import train_test_split

#for i, l in enumerate(tr_df['AdoptionSpeed'].values):
#    y_train[i,l] = 1
#for i, l in enumerate(val_df['AdoptionSpeed'].values):
#    y_valid[i,l] = 1
# train_df, noused_df = train_test_split(train_df, test_size = 0.6, random_state = 23)
tr_df, val_df = train_test_split(train_df, test_size = 0.33, random_state = 23)

In [179]:
# from keras.utils.np_utils import to_categorical

In [180]:
tr_df['AdoptionSpeed'].values.shape

(10045,)

In [181]:
tr_df

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
13507,2,"Missy, Bobtailed Kitten",4,266,249,2,1,3,7,2,...,1,1,-0.271132,41326,5aca30a29e8290093fee2a334d65d835,0,"If you love bob-tailed kitties, then how about...",4693e646e,14.0,1
10594,2,"Sneaky, Playful, Calmie, Adventura",2,266,0,3,4,5,6,2,...,1,4,-0.271132,41326,bd646cb96f00a7c5bff955a4084872fc,0,"Their mother came to us few months ago, she wa...",45afc38e4,8.0,3
1017,2,Wiki,3,266,0,1,4,7,0,2,...,1,1,-0.271132,41326,7b107e61f7d24a581e46e0abc4d2a7a7,0,Found this cutie at the Putrajaya Sentral car ...,69262e404,1.0,2
6321,1,Kruppy,1,307,0,1,1,3,7,2,...,1,1,-0.271132,41326,d4a8b5e64cd89e2f055b505383a8ee7a,0,[On trial adoption] Pics 1-5 with credits to O...,df17a57ae,6.0,1
12323,2,NaN,2,266,0,3,3,6,7,2,...,1,3,-0.271132,41401,abe54643224ddbb5c199cd46fbca0392,0,3 cute and playful kitties 2 months old for fr...,08ee62d43,2.0,3
8052,1,Xiao Ke,6,307,0,1,2,0,0,2,...,2,1,-0.271132,41327,4e9f9a3804795b86c27dd86f7fe05029,0,A dog that dumped by owner at market. Pls gv h...,87a679d31,1.0,4
5947,2,Little Red Bean,2,265,265,2,6,7,0,1,...,1,1,-0.271132,41401,93dfc7e5d65f9772958c411eb3a8b53f,0,Little Red Bean is still waiting for her forev...,f205b0649,13.0,3
13732,1,DELIGHTFUL PUPPIES!,3,307,0,3,1,2,5,2,...,1,4,-0.271132,41401,9dc662ce3243365c173dc9029aad6989,0,Here are 4 beautiful puppies for adoption! The...,a07a65495,1.0,4
987,1,--Shiro--,9,307,0,1,7,0,0,2,...,1,1,-0.271132,41327,c2e496d5b05dc9771f234e168465b389,0,A white mixed breed dog looking for adoption. ...,9c580ca06,1.0,4
9366,2,Max And Bobby,3,265,0,3,1,2,7,1,...,1,2,-0.271132,41401,1004c35eccb48a99a16726b45f0d7baa,0,"dua2 ni adik beradik..comel,sihat and cergas.....",fecec878e,2.0,4


In [182]:
y_train = tr_df['AdoptionSpeed'].values / 4
y_valid = val_df['AdoptionSpeed'].values / 4

In [183]:
y_train = np_utils.to_categorical(tr_df['AdoptionSpeed'], num_classes=5)
y_valid = np_utils.to_categorical(val_df['AdoptionSpeed'], num_classes=5)

In [184]:
X_train = get_input_features(tr_df)
X_valid = get_input_features(val_df)
X_test = get_input_features(test_df)

ValueError: `sequences` must be a list of iterables. Found non-iterable: nan

In [185]:
len(X_train)

19

In [186]:
hist = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), batch_size = 256, epochs = 10, verbose = True)

Train on 10045 samples, validate on 4948 samples
Epoch 1/10


InvalidArgumentError: indices[0,0] = 7 is not in [0, 7)
	 [[{{node embedding_138/embedding_lookup}} = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_5/Adam/Assign_23"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_138/embeddings/read, embedding_138/Cast, training_5/Adam/gradients/embedding_138/embedding_lookup_grad/concat/axis)]]

In [187]:

# zipcodes = tr_df[uncontinuous].value_counts().keys().tolist()
# counts = tr_df[uncontinuous].value_counts().tolist()

# 	# loop over each of the unique zip codes and their corresponding
# 	# count
# for (zipcode, count) in zip(zipcodes, counts):
# 		# the zip code counts for our housing dataset is *extremely*
# 		# unbalanced (some only having 1 or 2 houses per zip code)
# 		# so let's sanitize our data by removing any houses with less
# 		# than 25 houses per zip code
# 	if count < 25:
# 		idxs = tr_df[tr_df[uncontinuous] == zipcode].index
# 		tr_df.drop(idxs, inplace=True)
        
# zipBinarizer = LabelBinarizer().fit(tr_df[uncontinuous])
# trainCategorical = zipBinarizer.transform(tr_df[uncontinuous])

# print("----------------------------")
# print(tr_df)
continuous = ['Type','Age','Breed1','Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 
          'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized','Health', 'Fee','Quantity','VideoAmt','PhotoAmt']
cs = StandardScaler();
# LABEL = 'AdoptionSpeed'
# tr_df = tr_df.drop([LABEL], axis=1);
# val_df = val_df.drop([LABEL], axis=1);
# test_df = test_df.drop([LABEL], axis=1);
LABEL = 'RescuerID'
tr_df = tr_df.drop([LABEL], axis=1);
val_df = val_df.drop([LABEL], axis=1);
test_df = test_df.drop([LABEL], axis=1);
LABEL = 'Description'
tr_df = tr_df.drop([LABEL], axis=1);
val_df = val_df.drop([LABEL], axis=1);
test_df = test_df.drop([LABEL], axis=1);
LABEL = 'PetID'
tr_df = tr_df.drop([LABEL], axis=1);
val_df = val_df.drop([LABEL], axis=1);
test_df = test_df.drop([LABEL], axis=1);
LABEL = 'Name'
tr_df = tr_df.drop([LABEL], axis=1);
val_df = val_df.drop([LABEL], axis=1);
test_df = test_df.drop([LABEL], axis=1);
trainContinuous = cs.fit_transform(tr_df[continuous])
trainContinuous2 = cs.fit_transform(val_df[continuous])
trainContinuous3 = cs.fit_transform(test_df[continuous])

/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/zekunzhao/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataCon

In [ ]:
print(trainContinuous)

In [188]:
x_train = np.hstack([trainContinuous])
x_valid = np.hstack([trainContinuous2])
x_test = np.hstack([trainContinuous3])
x_train = tr_df[continuous]
x_valid = val_df[continuous]
model = Sequential([
    Dense(64, input_dim=18),
    Activation('relu'),
    Dense(96),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(5),
    Activation('sigmoid'),
])
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# We add metrics to get more results you want to see
#categorical_crossentropy--mse
model.compile(optimizer=rmsprop,
              loss='mse',
              metrics=['accuracy'])
model.fit(x_train, y_train,validation_data = (x_valid,y_valid),epochs=260, batch_size=10)

Train on 10045 samples, validate on 4948 samples
Epoch 1/260
10045/10045 [==============================] - 3s 327us/step - loss: 0.2154 - acc: 0.2274 - val_loss: 0.1640 - val_acc: 0.2122
Epoch 2/260
10045/10045 [==============================] - 2s 189us/step - loss: 0.1555 - acc: 0.2764 - val_loss: 0.1536 - val_acc: 0.2823
Epoch 3/260
10045/10045 [==============================] - 2s 181us/step - loss: 0.1536 - acc: 0.2996 - val_loss: 0.1519 - val_acc: 0.3068
Epoch 4/260
10045/10045 [==============================] - 2s 180us/step - loss: 0.1521 - acc: 0.3021 - val_loss: 0.1512 - val_acc: 0.2941
Epoch 5/260
10045/10045 [==============================] - 2s 181us/step - loss: 0.1509 - acc: 0.3194 - val_loss: 0.1493 - val_acc: 0.3159
Epoch 6/260
10045/10045 [==============================] - 2s 184us/step - loss: 0.1496 - acc: 0.3301 - val_loss: 0.1509 - val_acc: 0.3161
Epoch 7/260
10045/10045 [==============================] - 2s 182us/step - loss: 0.1489 - acc: 0.3406 - val_loss: 0.1

KeyboardInterrupt: 

In [ ]:
# print('\nTesting ------------')
# # Evaluate the model with the metrics we defined earlier
# loss, accuracy = model.evaluate(x_test, y_test)

# print('test loss: ', loss)
# print('test accuracy: ', accuracy)


In [ ]:
#y_pred2 = np.argmax(y_pred,axis = 1)
#y_pred2.shape

In [ ]:
tr_df